In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv())  
api_key = os.getenv("GOOGLE_API_KEY") 
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-05-20")

response = llm.invoke("Olá Gemini") 
print(response)

content='Olá! Como posso ajudar?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'models/gemini-2.5-flash-preview-05-20', 'safety_ratings': []} id='run--1b10e81d-32e7-423f-bb5d-63dc656a91e9-0' usage_metadata={'input_tokens': 3, 'output_tokens': 6, 'total_tokens': 172, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 163}}


In [36]:
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain.agents import create_tool_calling_agent
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain.prompts import MessagesPlaceholder

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

from pydantic import BaseModel, Field

import redis
from datetime import datetime
import requests
import os


REDIS_URL = os.getenv("REDIS_URL")
EVOLUTION_URL = os.getenv("EVOLUTION_URL")
INSTANCE_ID = os.getenv("EVOLUTION_INSTANCE")
EVOLUTION_TOKEN = os.getenv("EVOLUTION_APIKEY")

api_key = os.getenv("OPENAI_API_KEY")
api_key_gemini = os.getenv("GOOGLE_API_KEY")

#-------------------------------------------------Functions---------------------------------------------------------------

def obter_hora_e_data_atual():
    agora = datetime.now()
    return agora.strftime("%d-%m-%Y - %H:%M:%S")

def get_memory_for_user(whatsapp):
    memory = RedisChatMessageHistory(
        session_id=whatsapp, 
        url=REDIS_URL)
    return ConversationBufferMemory(return_messages=True, memory_key="memory", chat_memory=memory)

#------------------------------------------------Webhooks-------------------------------------------------------------------


user_id = "John Santos"
user_number = "5541996143338"
data_atual = obter_hora_e_data_atual()
memoria = get_memory_for_user(user_id)

    #--------------------------------------------------Tools----------------------------------------------------------
    
class Enviar_Evolution(BaseModel):
    texto: str = Field(description="Mensagem a ser enviada")

@tool(args_schema=Enviar_Evolution)
def enviar_msg(texto: str):
    """Envia uma mensagem de whatsapp ao usuario"""
    url = f"{EVOLUTION_URL}/message/sendText/{INSTANCE_ID}"
    headers = {
        "Content-Type": "application/json", 
        "apikey": EVOLUTION_TOKEN
    }
    payload = {
        "number": "5541996143338", 
        "text": texto, 
        "delay": 800
    }

    responsta = requests.post(url, headers=headers, json=payload)
    responsta.raise_for_status()

    return responsta.json()

@tool
def excluir_memoria():
    """Exclui a memoria da conversa quando solicitado"""
    r = redis.from_url(REDIS_URL, decode_responses=True)
    session_prefix = f"message_store:{user_id}"
    chaves = r.keys(session_prefix + "*")
    for chave in chaves:
        r.delete(chave)

tools = [enviar_msg, excluir_memoria]

#--------------------------------------------------Tools----------------------------------------------------------

prompt = ChatPromptTemplate.from_messages([
    ("system", f"""
        Você é Morpheus, um assistente disruptivo. A data atual é {data_atual}.
        Não use asteriscos, listas devem começar com '-'.
    """),
    MessagesPlaceholder(variable_name="memory"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])


LLM_PROVIDER = "gemini" 

if LLM_PROVIDER == "openai":
    llm = ChatOpenAI(model="gpt-4o", openai_api_key=api_key, temperature=0.0)
    tools_json = [convert_to_openai_function(t) for t in tools]
    pass_through = RunnablePassthrough.assign(agent_scratchpad=lambda x: format_to_openai_function_messages(x["intermediate_steps"]))
    chain = pass_through | prompt | llm.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

elif LLM_PROVIDER == "gemini":
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.0)
    chain = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=chain,
    memory=memoria,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True
)

resposta = agent_executor.invoke({"input": "qual foi a ultima mensagem que lhe enviei?"})
print(resposta)





> Entering new AgentExecutor chain...


ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 Unable to submit request because it has an empty text parameter. Add a value to the parameter and try again. Learn more: https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/gemini

# Gpt

In [ ]:

















from fastapi import FastAPI, Request, Form, HTTPException
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles

from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain.agents import create_tool_calling_agent
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain.prompts import MessagesPlaceholder

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

from dotenv import load_dotenv, find_dotenv
from pydantic import BaseModel, Field

import openai
import uvicorn
import redis
import base64
import tempfile
from datetime import datetime
import requests
import os

app = FastAPI()
templates = Jinja2Templates(directory="templates")
app.mount("/static", StaticFiles(directory="static"), name="static")

REDIS_URL = os.getenv("REDIS_URL")
EVOLUTION_URL = os.getenv("EVOLUTION_URL")
INSTANCE_ID = os.getenv("EVOLUTION_INSTANCE")
EVOLUTION_TOKEN = os.getenv("EVOLUTION_APIKEY")

api_key = os.getenv("OPENAI_API_KEY")
api_key_gemini = os.getenv("GOOGLE_API_KEY")

#-------------------------------------------------Functions---------------------------------------------------------------

def obter_hora_e_data_atual():
    agora = datetime.now()
    return agora.strftime("%d-%m-%Y - %H:%M:%S")

def get_memory_for_user(whatsapp):
    memory = RedisChatMessageHistory(
        session_id=whatsapp, 
        url=REDIS_URL)
    return ConversationBufferMemory(return_messages=True, memory_key="memory")

#------------------------------------------------Webhooks-------------------------------------------------------------------


user_id = "John Santos"
user_number = "5541996143338"
data_atual = obter_hora_e_data_atual()
memoria = get_memory_for_user(user_id)

    #--------------------------------------------------Tools----------------------------------------------------------
    
class Enviar_Evolution(BaseModel):
    texto: str = Field(description="Mensagem a ser enviada")

@tool(args_schema=Enviar_Evolution)
def enviar_msg(texto: str):
    """Envia uma mensagem de whatsapp ao usuario"""
    url = f"{EVOLUTION_URL}/message/sendText/{INSTANCE_ID}"
    headers = {
        "Content-Type": "application/json", 
        "apikey": EVOLUTION_TOKEN
    }
    payload = {
        "number": "5541996143338", 
        "text": texto, 
        "delay": 800
    }

    responsta = requests.post(url, headers=headers, json=payload)
    responsta.raise_for_status()

    return responsta.json()

@tool
def excluir_memoria():
    """Exclui a memoria da conversa quando solicitado"""
    r = redis.from_url(REDIS_URL, decode_responses=True)
    session_prefix = f"message_store:{user_id}"
    chaves = r.keys(session_prefix + "*")
    for chave in chaves:
        r.delete(chave)

tools = [enviar_msg, excluir_memoria]

#--------------------------------------------------Tools----------------------------------------------------------
def format_intermediate_steps_text(steps):
    scratchpad = ""
    for action, observation in steps:
        scratchpad += f"Action: {action.tool} | Input: {action.tool_input}\n"
        scratchpad += f"Observation: {observation}\n"
    return scratchpad.strip()



prompt = ChatPromptTemplate.from_messages([
    ("system", f"""
        Você é Morpheus, um assistente disruptivo. A data atual é {data_atual}.
        Não use asteriscos, listas devem começar com '-'.
        {agent_scratchpad}
    """),
    MessagesPlaceholder(variable_name="memory"),
    ("user", "{input}")
])


# --- Formatador compatível com OpenAI ou Gemini ---


# --- Escolha do modelo e formatação de acordo com o provedor ---
LLM_PROVIDER = "gemini"  # ou "openai"

if LLM_PROVIDER == "openai":
    llm = ChatOpenAI(model="gpt-4o", openai_api_key=api_key, temperature=0.0)
    tools_json = [convert_to_openai_function(t) for t in tools]
    
    pass_through = RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_to_openai_function_messages(x["intermediate_steps"])
    )
    
    chain = pass_through | prompt | llm.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

elif LLM_PROVIDER == "gemini":
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-05-20", google_api_key=api_key_gemini)
    
    pass_through = RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_intermediate_steps_text(x["intermediate_steps"])
    )
    
    chain = pass_through | prompt | llm.bind_tools(tools)



#  elif LLM_PROVIDER == "gemini":
#      chain = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=chain,
    memory=memoria,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True
)

resposta = agent_executor.invoke({"input": "ola tudo bem?"})
print(resposta)

